In [ ]:
!pip install numpy scipy umap-learn matplotlib


In [ ]:
import numpy as np
import umap
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist
import sys

# Function to generate PRNG data
def generate_prng_data(num_samples, dimensions):
    return np.random.random((num_samples, dimensions))

# Function to measure pairwise distances
def measure_pairwise_distances(data):
    distances = pdist(data, metric='euclidean')
    return distances

# Function to check for near-duplicate points (cycling or periodicity)
def check_for_duplicates(data, tolerance=1e-6):
    unique_data = np.unique(np.round(data, decimals=6), axis=0)
    return len(data) - len(unique_data)

# Function to apply UMAP for dimensionality reduction
def apply_umap(data, n_neighbors=15, min_dist=0.1):
    reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, random_state=42)
    embedding = reducer.fit_transform(data)
    return embedding

# Function to visualize the data after UMAP
def visualize_umap(embedding, title):
    plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5)
    plt.title(title)
    plt.show()

# Function to detect when PRNG starts to fail
def detect_near_collapse(pairwise_distances, duplicate_count, threshold_distance=0.1, threshold_duplicates=10):
    # Check if pairwise distances are clustering (i.e., very small distances start to dominate)
    distance_collapsing = np.mean(pairwise_distances) < threshold_distance
    # Check if duplicate points are appearing in large numbers
    duplicate_collapsing = duplicate_count >= threshold_duplicates
    return distance_collapsing or duplicate_collapsing

# Main function to run the experiment
def run_experiment(num_samples=1000, start_dim=2, max_dim=sys.maxsize, collapse_threshold=0.1):
    dim = start_dim
    collapse_detected = False
    iteration = 1
    while not collapse_detected and dim < max_dim:
        print(f"Iteration {iteration}: Analyzing {dim}-dimensional data")

        # Step 1: Generate PRNG Data
        data = generate_prng_data(num_samples, dim)

        # Step 2: Measure Pairwise Distances
        distances = measure_pairwise_distances(data)
        avg_distance = np.mean(distances)
        print(f"Average pairwise distance in {dim} dimensions: {avg_distance:.5f}")

        # Step 3: Check for duplicates (to detect periodicity)
        duplicate_count = check_for_duplicates(data)
        print(f"Number of near-duplicate points in {dim} dimensions: {duplicate_count}")

        # Step 4: Apply UMAP for visualization
        embedding = apply_umap(data)
        visualize_umap(embedding, f"UMAP projection of {dim}D data to 2D")

        # Step 5: Check for collapse
        collapse_detected = detect_near_collapse(distances, duplicate_count, threshold_distance=collapse_threshold)
        if collapse_detected:
            print(f"Collapse detected at {dim} dimensions!")
            break

        print("------------\n")

        # Step 6: Exponentially increase the number of dimensions
        dim *= 2
        iteration += 1

# Run the experiment with exponentially increasing dimensions
run_experiment()


In [ ]:
import numpy as np
from scipy.stats import entropy

# Function to generate PRNG data at 1 trillion dimensions
def generate_prng_data(num_samples, dimensions):
    return np.random.random((num_samples, dimensions))

# Function to measure variance across dimensions
def measure_variance(data):
    return np.var(data, axis=0)

# Function to calculate entropy
def calculate_entropy(data):
    hist, _ = np.histogram(data, bins=50, density=True)
    return entropy(hist)

# Main function to run the analysis at 1 trillion dimensions
def analyze_1_trillion_dimensions(num_samples=1000, dimensions=int(1e8)):
    print(f"Analyzing PRNG data with {dimensions} dimensions and {num_samples} samples")

    # Step 1: Generate PRNG Data
    data = generate_prng_data(num_samples, dimensions)
    print("Data generation complete.")

    # Step 2: Measure Variance Across Dimensions
    variances = measure_variance(data)
    avg_variance = np.mean(variances)
    print(f"Average variance in {dimensions} dimensions: {avg_variance:.5f}")

    # Step 3: Calculate Entropy
    entropy_value = calculate_entropy(data.flatten())
    print(f"Entropy in {dimensions} dimensions: {entropy_value:.5f}")

    print("Analysis complete.")

# Run the analysis with 1 trillion dimensions
analyze_1_trillion_dimensions()


In [1]:
import numpy as np

# Function to detect the period of a PRNG by finding repetitions
def detect_prng_period(num_values, chunk_size=100000):
    seen = set()  # Set to track unique generated values
    prng = np.random.default_rng()  # Modern PRNG from numpy
    
    for i in range(0, num_values, chunk_size):
        values = prng.random(chunk_size)  # Generate a chunk of random values
        for value in values:
            if value in seen:
                print(f"Period detected after generating {i} values!")
                return
            seen.add(value)
    print(f"No period detected after {num_values} values.")
    
# Run the test for a large number of values to see when/if repetition starts
detect_prng_period(num_values=1000000000)  # Test with a large number of values


Period detected after generating 82400000 values!


In [ ]:
from scipy.stats import chisquare
import numpy as np
# Function to perform a Chi-Square test for uniform distribution
def test_uniformity(num_samples, num_bins=10):
    prng = np.random.default_rng()
    samples = prng.random(num_samples)
    
    # Create a histogram of the random samples
    hist, bin_edges = np.histogram(samples, bins=num_bins, density=False)
    
    # Expected count is uniform for a truly random generator
    expected_count = num_samples / num_bins
    chi_stat, p_value = chisquare(hist, [expected_count] * num_bins)
    
    print(f"Chi-Square statistic: {chi_stat}, p-value: {p_value}")
    if p_value < 0.05:
        print("Significant deviation from uniformity detected!")
    else:
        print("No significant deviation from uniformity.")
    
# Run the test with a large number of samples
test_uniformity(num_samples=10000000000)
